<a href="https://colab.research.google.com/github/chiangtinhung-rgb/Matcha/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C_%E8%8B%B1%E6%96%87%E5%AD%B8%E6%B8%AC%E5%91%BD%E9%A1%8C%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安裝依賴套件
!pip install -q gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil \
               pdfplumber jieba scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.2 MB/s eta 0:00:00


In [2]:
# 基礎設置與 API 連接

import os, time, uuid, re, json, random
from datetime import datetime as dt
from dateutil.tz import gettz
import pandas as pd
import numpy as np
import gradio as gr
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import google.generativeai as genai
from google.generativeai.types import GenerationConfig

# Google Sheets & Auth 相關套件
from google.colab import auth, userdata
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth import default

In [3]:
# 新增 TF-IDF 相關套件
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


In [4]:
# --- 1. Google 權限驗證 ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [5]:
# --- 2. Gemini API 配置 ---
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('GERE')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-flash')

In [6]:
# --- 3. 全域變數設定  ----
# 請確保此連結權限已開放給你的帳號編輯
SHEET_URL = "https://docs.google.com/spreadsheets/d/1dRVg93FmRuDHEyzkSlAbW8vBz86BWrgSs5YPl73P8to/edit?gid=2104142992#gid=2104142992"
WORKSHEET_NAME_TFIDF = "歷屆試題熱詞_TFIDF"
WORKSHEET_NAME_GEMINI = "歷屆試題分析_Gemini"
TIMEZONE = "Asia/Taipei"

In [7]:
# 初始化 Google Sheets 分頁與欄位
def init_google_sheets(sheet_url):
    try:
        sh = gc.open_by_url(sheet_url)
        print(f"成功開啟試算表: {sh.title}")
    except Exception as e:
        print(f"無法開啟試算表，請確認 URL 與權限: {e}")
        return

    # 定義我們需要的資料表結構 (根據你的需求 1~6 點規劃)
    required_sheets = {
        "歷屆試題熱詞_TFIDF": [
            "年份", "大題", "單字", "詞頻(Count)", "TF-IDF值", "更新時間"
        ],
        "歷屆試題分析_Gemini": [
            "年份", "大題", "洞察摘要_1", "洞察摘要_2", "洞察摘要_3", "50字結論", "更新時間"
        ]
    }

    # 檢查並建立分頁
    for worksheet_name, headers in required_sheets.items():
        try:
            ws = sh.worksheet(worksheet_name)
            print(f"分頁 [{worksheet_name}] 已存在，檢查欄位...")

            # 檢查第一列是否為空，若為空則補上欄位
            existing_headers = ws.row_values(1)
            if not existing_headers:
                ws.append_row(headers)
                print(f" -> 已補上欄位名稱。")
        except gspread.WorksheetNotFound:
            print(f"分頁 [{worksheet_name}] 不存在，正在建立...")
            ws = sh.add_worksheet(title=worksheet_name, rows=1000, cols=10)
            ws.append_row(headers)
            print(f" -> 分頁建立完成。")

    print("\n所有指定分頁與欄位結構已確認就緒！")

# 執行初始化
init_google_sheets(SHEET_URL)

成功開啟試算表: 學測英文命題系統
分頁 [歷屆試題熱詞_TFIDF] 已存在，檢查欄位...
分頁 [歷屆試題分析_Gemini] 已存在，檢查欄位...

所有指定分頁與欄位結構已確認就緒！


In [8]:
# --- 輔助函數: 分詞與清理 ---
def preprocess_text(text):
    """使用 jieba 進行中文分詞 (學測英文試題中可能包含中文說明) 並清理標點符號。"""
    # 僅保留英文單字 (a-z, A-Z) 和空格
    # 假設輸入的試題內容主要是英文，我們將專注於提取英文單詞
    english_only = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # 過濾掉常見的英文停用詞 (Stop Words)，但此處我們不進行分詞
    # 因為英文的 TF-IDF 不需要像中文一樣用 jieba 來分詞
    return english_only.lower()

# --- 核心函數: TF-IDF 分析與寫入 ---
def analyze_and_write_tfidf(raw_text, year, topic, top_n=20):
    """
    1. 對上傳的試題文本進行 TF-IDF 分析。
    2. 將前 N 個熱詞及數值寫入 Google Sheet。
    """
    # 預處理文本 (轉小寫, 去除非字母字符)
    processed_text = preprocess_text(raw_text)

    # 英文的 TF-IDF Vectorizer
    # stop_words='english' 會自動過濾常見詞彙，如 the, a, is, of, in 等
    vectorizer = TfidfVectorizer(stop_words='english', token_pattern=r'\b[a-zA-Z]{3,}\b') # 限制單字長度至少為3

    # 計算詞頻和 TF-IDF
    try:
        tfidf_matrix = vectorizer.fit_transform([processed_text])
    except ValueError:
        return "錯誤：文本內容不足或無有效英文單字可供分析。", pd.DataFrame()

    feature_names = vectorizer.get_feature_names_out()

    # 統計詞頻 (Count)
    counts = np.sum(tfidf_matrix.toarray() > 0, axis=0) # 簡化詞頻，這裡實際是文件頻次
    # 為了更準確的'詞頻(Count)'，我們需要一個簡單的詞彙計數
    word_counts = pd.Series(processed_text.split()).value_counts()

    # 獲取 TF-IDF 值
    tfidf_scores = tfidf_matrix.max(axis=0).toarray().flatten()

    df_tfidf = pd.DataFrame({'單字': feature_names, 'TF-IDF值': tfidf_scores})

    # 合併詞頻
    df_tfidf['詞頻(Count)'] = df_tfidf['單字'].apply(lambda x: word_counts.get(x, 0))

    # 排序並選取前 N
    df_tfidf = df_tfidf.sort_values(by=['TF-IDF值', '詞頻(Count)'], ascending=[False, False]).head(top_n)

    # 準備寫入 Google Sheets 的資料
    now = dt.now(gettz(TIMEZONE)).strftime('%Y-%m-%d %H:%M:%S')
    df_tfidf['年份'] = year
    df_tfidf['大題'] = topic
    df_tfidf['更新時間'] = now

    # 重新排序欄位以匹配 Google Sheet
    df_to_write = df_tfidf[['年份', '大題', '單字', '詞頻(Count)', 'TF-IDF值', '更新時間']]

    # 寫入 Google Sheet (歷屆試題熱詞_TFIDF)
    try:
        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME_TFIDF)
        # 使用 gspread_dataframe 的 append 模式
        # 注意: set_with_dataframe 會覆蓋，我們使用 append 模式寫入新資料
        existing_data = get_as_dataframe(ws, header=1, evaluate_formulas=True)
        combined_df = pd.concat([existing_data, df_to_write], ignore_index=True)
        set_with_dataframe(ws, combined_df.fillna(''), row=1, col=1)

        return f"✅ 已成功分析並將 {len(df_to_write)} 個熱詞寫入 '{WORKSHEET_NAME_TFIDF}'。", df_to_write

    except Exception as e:
        return f"❌ 寫入 Google Sheets 失敗: {e}", df_to_write

In [9]:
# --- 核心函數: Gemini 分析與寫入 ---

import json # 確保 json 已匯入

def analyze_with_gemini_and_write(year, topic, hot_words_df):
    """
    根據熱詞呼叫 Gemini API 生成洞察摘要與結論，並寫入 Google Sheet。
    """
    if hot_words_df.empty or '單字' not in hot_words_df.columns:
        error_json = {"status": "error", "message": "無有效熱詞資料，無法執行 Gemini 分析。請先成功執行 TF-IDF。"}
        return "錯誤：無熱詞資料可供分析。", json.dumps(error_json, ensure_ascii=False, indent=2)

    hot_words_list = hot_words_df['單字'].astype(str).tolist()

    prompt = f"""
    任務：你是學測命題趨勢專家。請根據「{year} 年 {topic}」的熱門單字，生成命題洞察分析。
    熱門單字: {', '.join(hot_words_list)}

    請嚴格依照以下格式輸出：
    {{
        "Insight_1": "第一句專業洞察摘要 (35個中文字)",
        "Insight_2": "第二句專業洞察摘要 (35個中文字)",
        "Insight_3": "第三句專業洞察摘要 (35個中文字)",
        "Conclusion_50": "一段總結分析結論 (在80個中文字左右)"
    }}
    """

    try:
        # 🎯 修正: 定義 GenerationConfig 物件
        generation_config = GenerationConfig(
            response_mime_type="application/json"
        )

        response = model.generate_content(
            prompt,
            generation_config=generation_config # <-- 參數名稱修正為 generation_config
        )

        # ... (JSON 解析和寫入 Sheets 邏輯保持不變)
        if not response.text or response.text.strip() == '':
            error_json = {"status": "error", "message": "Gemini API 回應內容為空。"}
            return f"❌ Gemini API 回應內容為空。", json.dumps(error_json, ensure_ascii=False, indent=2)

        try:
            analysis_data = json.loads(response.text)
        except json.JSONDecodeError as e:
            error_json = {
                "status": "error",
                "message": f"Gemini API 返回內容不是有效的 JSON 格式: {e}",
                "raw_response_snippet": response.text[:300] + "..."
            }
            return f"❌ Gemini API 返回內容解析失敗。", json.dumps(error_json, ensure_ascii=False, indent=2)

        now = dt.now(gettz(TIMEZONE)).strftime('%Y-%m-%d %H:%M:%S')
        data_to_write = {
            '年份': year, '大題': topic, '洞察摘要_1': analysis_data.get('Insight_1', 'N/A'),
            '洞察摘要_2': analysis_data.get('Insight_2', 'N/A'), '洞察摘要_3': analysis_data.get('Insight_3', 'N/A'),
            '50字結論': analysis_data.get('Conclusion_50', 'N/A'), '更新時間': now
        }

        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME_GEMINI)
        existing_data = get_as_dataframe(ws, header=1, evaluate_formulas=True)
        new_row_df = pd.DataFrame([data_to_write])
        combined_df = pd.concat([existing_data, new_row_df], ignore_index=True)
        set_with_dataframe(ws, combined_df.fillna(''), row=1, col=1)

        return f"✅ 已成功生成 Gemini 分析並寫入 '{WORKSHEET_NAME_GEMINI}'。", json.dumps(analysis_data, ensure_ascii=False, indent=2)

    except Exception as e:
        error_json = {"status": "fatal_error", "message": f"發生致命錯誤: {e}"}
        return f"❌ Gemini API 呼叫或寫入 Sheets 失敗: {e}", json.dumps(error_json, ensure_ascii=False, indent=2)

In [10]:
# --- 命題參數與選項定義 ---

# 1. 年份與題型篩選 (給分析用)
YEARS = ["114", "113", "112", "111"]
TOPICS_ANALYSIS = ["詞彙題", "綜合測驗", "文意選填", "篇章結構", "閱讀測驗", "混合題"]

# 2. 命題配置
QUESTION_CONFIG = {
    "詞彙題": {"count": 10, "length": "N/A", "difficulty": ["易", "中", "難"],
             "structure_details": "生成 10 題。每題為一句獨立句子挖空。",
             "themes": ["日常生活", "社會現象與議題", "自然與環境", "情感與心理狀態", "職場與人際", "科技與創新", "社會文化", "個人價值觀", "社會與個人經驗", "學術與工作"]},

    "綜合測驗": {"count": 10, "length": "短文(2篇)", "difficulty": ["中", "難"],
             "structure_details": "生成 2 篇文章，每篇 3-4 段，共 10 題 (各 5 題)",
             "themes": ["科技與工程", "歷史發展", "自然科學", "文化", "環境保護", "社會文化與現象", "歷史與科技發展", "人物介紹", "社會創新", "科普知識", "社會人文"]},

    "文意選填": {"count": 10, "length": "中短文(1篇)", "difficulty": ["中"],
             "structure_details": "生成 1 篇 4-5 段文章，提供 10 個空格與選項供選填。",
             "themes": ["歷史與發展", "電影與娛樂產業", "人物與傳記", "健康與生活", "科普知識", "心理學", "個人情感", "災難與療癒", "文化遺產歷史", "藝術建築"]},

    "篇章結構": {"count": 5, "length": "中短文(1篇)", "difficulty": ["中", "難"],
             "structure_details": "生成 1 篇 4-5 段文章，移除 4 句，要求從5句選項中選出 4 句填入。",
             "themes": ["國際文化", "歷史起源", "社會現象", "心理學行為", "科普知識", "健康醫療", "社會觀察", "商業模式", "旅遊住宿"]},

    "閱讀測驗": {"count": 12, "length": "長文(3篇)", "difficulty": ["中", "難"],
             "structure_details": "生成 3 篇 各4-5 段文章，共 12 題 (每篇 4 題)。",
             "themes": ["工程與工業設計", "生物學與醫學研究", "歷史文化", "環境與生態", "科普知識", "社會議題", "科學與科技", "藝術與歷史", "環境與創新", "科技發展與歷史", "社會心理學"]},

    "混合題組": {"count": 3, "length": "雙文本/論壇(1組)", "difficulty": ["中", "難"],
             "structure_details": "生成 1 組題組，包含 3 題：單字填空/片語回答/綜合文意多選 各 1 題。",
             "themes": ["人物與傳記", "體育與勵志故事", "科技與生活", "消費觀念", "健康與飲食", "科學研究", "生物行為", "社會議題論述", "論壇討論", "心理學"]},

    "中譯英": {"count": 2, "length": "N/A", "difficulty": ["中", "難"],
             "structure_details": "生成兩句連貫的中文句子進行翻譯。",
             "themes": ["生活素養", "社會議題", "論述性與普世價值", "台灣社會文化", "時事議題", "學術知識"]},

    "英文作文": {"count": 1, "length": "N/A", "difficulty": ["中", "難"],
             "structure_details": "生成一篇要求兩段式的英文作文題目。",
             "themes": ["新興科技", "生活經驗與情境描寫", "議題探討型", "青少年生活與心理", "跨領域議題", "時事熱點"]}
}

# 3. ARTICLE_LENGTHS 變數
ARTICLE_LENGTHS = [
    "不限制",
    "250-300 字",
    "300-350 字",
    "350-400 字",
    "400 字以上"
]

TOPICS_GENERATION = list(QUESTION_CONFIG.keys()) # 作為命題的題型下拉選單
DIFFICULTIES = ["易", "中", "難"] # 難度選項

In [11]:
# get_topic_options 函數
def get_topic_options(topic_name):
    """根據選定的題型，動態返回該題型的主題選項清單和是否需要文章長度的提示。"""
    config = QUESTION_CONFIG.get(topic_name, {})
    themes = config.get("themes", [])

    # 判斷是否為不需要文章長度的題型
    if topic_name in ["詞彙題", "中譯英", "英文作文"]:
        # 這些題型不需要文章長度，長度選單在介面上會被禁用
        is_length_required = False
        length_hint = "N/A (不適用文章長度)"
    else:
        is_length_required = True
        length_hint = "文章長度設定" # 啟用文章長度下拉選單

    # 輸出：主題選單選項 (choices), 文章長度提示 (Textbox value)
    return gr.Dropdown(choices=themes, value=None), gr.Textbox(value=length_hint)

In [12]:
# --- generate_question 函數 ---
def generate_question(topic, difficulty, theme, article_length):
    """
    根據用戶選擇的參數，串接 Gemini API 生成完整的學測題目。
    """
    # 修正: 處理 Gradio 多選 (multiselect) 輸入，將列表轉換為逗號分隔的字符串
    if isinstance(theme, list):
        # 將列表中的主題用逗號和空格連接起來
        selected_themes = ", ".join(theme)
    elif isinstance(theme, str) and theme:
        selected_themes = theme
    else:
        # 如果用戶沒有選取主題，給予一個預設提示
        selected_themes = "未指定主題，請隨機選擇學測常見的主題類型。"

    config = QUESTION_CONFIG.get(topic, {})

    # 根據題型調整 Prompt 細節
    if topic == "詞彙題":
        theme_hint = f"從以下主題中隨機選 5 種作為出題素材: {', '.join(config['themes'])}"
        output_format_hint = f"請生成 {config['count']} 題詞彙選擇題。每題須包含：完整句子（含一個空格）、四個選項（A, B, C, D）、正確答案。"
    elif topic == "綜合測驗":
        theme_hint = f"請選取 {theme} 裡的 2 個主題作為文章主軸。"
        output_format_hint = f"請生成 2 篇短文，每篇 3-4 段，共 {config['count']} 題。每題皆為四選一的詞彙或文法選擇題，須提供選項與正確答案。"
    elif topic == "英文作文":
        theme_hint = f"選取主題: {theme}。"
        output_format_hint = "請根據作文出題策略（情境/圖片的複雜化，兩段式、多重任務提示結構），生成一個完整的作文題目要求（包含情境文字引導，若為雙圖片比較則用文字描述）。"

    # 💡 新增：獲取題目結構細節
    structure_details = config.get("structure_details", "無特定結構要求。")

    # 輔助函數: 獲取題型定義 (請確保這個輔助函數 get_topic_definition 在您的程式碼中存在)
    def get_topic_definition(topic_name):
        # 這是您在步驟 8 中定義的內容
        definitions = {
            "詞彙題": "給予一個句子，在句子中挖空，要求從四個選項中選出最符合句意和語法的單字 出10題。",
            "綜合測驗": "給予兩篇3-4段的文章，每篇在數個位置挖空5格，要求從四個選項中選出最合適的文法結構或詞彙，使文章語意連貫。",
            "文意選填": "給予一篇4-5段的文章，並提供十個選項（多為詞彙或片語），需將選項填入文章的空格中，使文意通順。選項會等於空格數。",
            "篇章結構": "給予一篇4-5段的文章，在數個段落中間或內部移除4個句子，並將這些句子打亂順序做成選項提供給考生，要求從5句選項中選出 4 句填入，使文章結構完整且語意連貫。",
            "閱讀測驗": "提供長度不等的文章3篇，每篇文章4-5段且後附4題問題，要求考生根據文章內容選出最合適的答案。",
            "混合題組": "提供引言+雙文本分析或6-7人的論壇討論 二擇一，包含 3 題：單字填空/片語回答/綜合文意多選 各 1 題要求考生統整資訊並用英文表達。",
            "中譯英": "給予兩句連貫的中文句子，要求考生將其翻譯成正確、流暢的英文。",
            "英文作文": "根據中文的文字引導、圖片提示(以英文描述呈現)，要求考生寫一篇兩段式的英文作文。"
        }
        return definitions.get(topic_name, "N/A")


    # 定義通用的命題 Prompt
    prompt = f"""
    任務：你是一位專業的學測英文命題委員。請根據指定的題型、難度與主題，使用你所掌握的台灣高中常用的學測，生成一組完整的學測題目。

    ---
    【命題要求】
    * **題型：** {topic}
    * **難度：** {difficulty} (請確保文章選材與單字難度符合此設定)
    * **文章長度：** {article_length}
    * **選材主題：** {selected_themes}  # 使用處理後的變數
    * **命題定義：** {get_topic_definition(topic)}

    ---

    請以 Markdown 格式清晰呈現題目內容，**不需要**生成詳解或答案，僅呈現題目本身。
    """

    try:
        response = model.generate_content(prompt)
        # 初始時 solution_output 給定提示
        return response.text, "請點擊「生成詳解」按鈕獲取答案與分析。"
    except Exception as e:
        # 如果 API 呼叫失敗，返回錯誤訊息
        return f"❌ 題目生成失敗，請檢查 API 金鑰或網路連線: {e}", "題目生成失敗，無法生成詳解。"

In [13]:
# 詳解與分析生成函數 (`generate_solution`)

def generate_solution(generated_question, topic):
    """
    呼叫 Gemini API 根據生成的題目，生成詳解、80字命題方向摘要和建議作答方向和作文範文。
    """
    # 構建 Prompt
    if topic == "英文作文":
        solution_type = "作文範文（約 260 字）"
        prompt_suffix = "請根據上述作文題目，生成一篇約 260 字的英文範文，並在最後提供一個 80 字的「命題方向摘要」和 80 字的建議作答方向。"
    else:
        solution_type = "完整詳解與命題摘要"
        prompt_suffix = "請提供此題目的完整解答（包含所有客觀題的正確選項與解釋），並在最後提供一個 80 字的「命題方向摘要」和 80 字的建議作答方向。"

    prompt = f"""
    任務：你是一位專業的英文教學專家。請針對以下題目，生成專業的 {solution_type}。

    ---
    【原始題目】
    {generated_question}
    ---

    {prompt_suffix}
    請使用 Markdown 格式清晰呈現。
    """

    try:
        response = model.generate_content(prompt)
        return response.text # 詳解生成成功
    except Exception as e:
        return f"❌ 詳解生成失敗: {e}"


In [14]:
# PDF 輸出函數
import os

def export_to_pdf(generated_text, filename="exam_paper.txt"):
    """
    將生成的題目文本儲存為 .txt 文件，並返回文件路徑供 Gradio 下載。

    注意：在 Gradio/Colab 環境中，直接生成複雜的 PDF (如 ReportLab) 可能會有額外依賴問題。
    此處先將內容輸出為文本文件 (.txt)，確保下載功能可用。
    """

    # 確保輸出目錄存在 (使用當前工作目錄)
    output_dir = os.getcwd()
    file_path = os.path.join(output_dir, filename)

    try:
        # 將文本內容寫入 .txt 文件 (使用 utf-8 編碼確保中文正確)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(generated_text)

        # Gradio 的 gr.File 元件需要接收檔案的路徑 (單一字串)
        return file_path

    except Exception as e:
        print(f"寫入檔案失敗: {e}")
        # 失敗時返回 None，Gradio 會顯示下載失敗
        return None

In [ ]:
# Gradio 介面定義與啟動

with gr.Blocks(title="學測英文命題系統") as iface:
    gr.Markdown("# 🎓 學測英文命題與分析系統")
    gr.Markdown("本系統整合 TF-IDF 熱詞分析與 Gemini API 進行命題洞察及自動出題。")

    # --- 標籤頁 1: 試題分析與熱詞洞察 ---
    with gr.Tab("試題分析與熱詞洞察"):
        gr.Markdown("## 📈 歷屆試題分析 (TF-IDF & Gemini) - 數據輸入與寫入 Sheets")

        with gr.Row():
            year_select_ana = gr.Dropdown(choices=YEARS, label="選擇分析年份", value="114")
            topic_select_ana = gr.Dropdown(choices=TOPICS_ANALYSIS, label="選擇分析大題", value="閱讀測驗")

        text_input = gr.Textbox(label="貼上試題文本內容 (請貼上單一大題的完整英文文本)", lines=10)
        analyze_button = gr.Button("🚀 執行 TF-IDF & Gemini 分析 (寫入 Google Sheets)")

        # 輸出區
        analysis_status = gr.Textbox(label="分析狀態/寫入結果", lines=2)
        hot_words_output = gr.Dataframe(label="TF-IDF 熱詞前 20 名", interactive=False, headers=['年份', '大題', '單字', '詞頻(Count)', 'TF-IDF值', '更新時間'])
        gemini_insight_output = gr.JSON(label="Gemini 洞察摘要與結論") # 使用 gr.JSON 來顯示結構化結果

        # 點擊按鈕觸發分析 (依序執行 TF-IDF -> Gemini)
        analyze_button.click(
            fn=analyze_and_write_tfidf,
            inputs=[text_input, year_select_ana, topic_select_ana],
            outputs=[analysis_status, hot_words_output],
            queue=False
        ).then(
            fn=analyze_with_gemini_and_write,
            # 這裡 hot_words_output 傳入的是上一步的結果，作為 Gemini 分析的輸入
            inputs=[year_select_ana, topic_select_ana, hot_words_output],
            outputs=[analysis_status, gemini_insight_output],
            queue=False
        )

   # --- 標籤頁 2: AI 自動命題區 (已修正) ---
    with gr.Tab("AI 自動命題區"):
        gr.Markdown("## 🤖 參數化命題生成")

        with gr.Row():
            topic_select_gen = gr.Dropdown(choices=TOPICS_GENERATION, label="命題題型", value="閱讀測驗")
            difficulty_select_gen = gr.Dropdown(choices=DIFFICULTIES, label="難易度", value="中")

        with gr.Row():
            # 文章長度下拉選單
            article_length_select = gr.Dropdown(
                choices=ARTICLE_LENGTHS,
                label="文章長度設定 (適用於文章題)",
                value="不限制",
                interactive=True,
                scale=1
            )

            # 主題選單
            initial_themes = QUESTION_CONFIG["詞彙題"]["themes"]
            theme_select_gen = gr.Dropdown(choices=initial_themes, label="選取命題主題 (可多選)", multiselect=True, scale=2)

        # ⚠️ 1. 修正: 必須在按鈕綁定前定義按鈕
        generate_button = gr.Button("✨ 生成學測題目", variant="primary")

        # ⚠️ 2. 修正: 必須定義 length_hint 才能讓 topic_select_gen.change 正常運作
        length_hint = gr.Textbox(label="題型固定長度提示", value=QUESTION_CONFIG["詞彙題"]["length"], interactive=False, visible=False)

        # 3. 動態更新主題選單和長度提示
        topic_select_gen.change(
            fn=get_topic_options,
            inputs=[topic_select_gen],
            outputs=[theme_select_gen, length_hint],
            queue=False
        )

        gr.Markdown("### 📜 題目呈現區")
        generated_question_output = gr.Textbox(label="生成結果 (Markdown 格式)", lines=25)

        # 詳解按鈕與輸出
        with gr.Row():
            solution_button = gr.Button("💡 生成詳解與命題方向摘要 (作文為範文)")
            # PDF 輸出元件 (使用正確的 File 元件)
            pdf_output = gr.File(label="下載生成的試卷 (.txt 模擬 PDF)", interactive=False, type="filepath")
            pdf_button = gr.Button("💾 輸出試卷檔案 (下載)")

        solution_output = gr.Textbox(label="詳解/範文與摘要", lines=15)

        # 4. 題目生成動作 (使用正確的輸入 article_length_select，且只定義一次)
        generate_button.click(
            fn=generate_question,
            inputs=[topic_select_gen, difficulty_select_gen, theme_select_gen, article_length_select],
            outputs=[generated_question_output, solution_output]
        )

        # 詳解生成動作
        solution_button.click(
            fn=generate_solution,
            inputs=[generated_question_output, topic_select_gen],
            outputs=[solution_output]
        )

        # PDF 輸出動作
        pdf_button.click(
            fn=export_to_pdf,
            inputs=[generated_question_output],
            outputs=pdf_output
        )

# 運行 Gradio 介面
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b1b710381b3f63d645.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
